# Creating a DuckDB database from a dataset

There are lots of options out there for creating SQL databases, but a very powerful and free option is to use [DuckDB](https://duckdb.org/). The advantage of DuckDB is that has an API for just about every programming language out there (Python, C, R, Java, etc.) so you can use it however you like. It also requires very little set up as you can query directly from a flat file like a CSV, or multiple files, or from a partitioned file system for better efficiency.

We'll use the Numerator dataset that we have been using on AWS/Athena. This dataset is also available on KLC in the form of parquet files.

## What is parquet?

> Apache Parquet is an open source, column-oriented data file format designed for efficient data storage and retrieval. It provides efficient data compression and encoding schemes with enhanced performance to handle complex data in bulk. Apache Parquet is designed to be a common interchange format for both batch and interactive workloads.
-- [DataBricks](https://www.databricks.com/glossary/what-is-parquet)

Essentially, parquet files are tabular just like a CSV file, except they are also (1) compressed and (2) preserve the data types of each column, which makes them easy to load into a database and maintain integrity.

## Understanding the file system

The Numerator dataset on KLC lives here:

```/kellogg/data/numerator/data/parquet-part```




In [1]:
ls -l /kellogg/data/numerator/data/parquet-part

total 98
drwxrwx--x+  2 wkt406 wkt406  4096 Jun 19 12:01 standard_nmr_feed_banner_table/
drwxrwx--x+  2 wkt406 wkt406 65536 Jun 20 15:53 standard_nmr_feed_fact_table/
drwxrwx--x+ 25 wkt406 wkt406  4096 Jun 30 12:36 standard_nmr_feed_item_table/
drwxrwx--x+  2 wkt406 wkt406 16384 Jun 19 12:12 standard_nmr_feed_people_attributes_table/
drwxrwx--x+ 13 wkt406 wkt406  4096 Jun 19 12:01 standard_nmr_feed_people_table/
drwxrwx--x+ 77 wkt406 wkt406 16384 Jun 19 12:12 standard_nmr_feed_static_table/


Each subdirectory contains all the data from a single table, saved as multiple parquet files.

### Using the partitions

You can tell which tables are partitioned based on the naming of subdirectories. In this case, the item table directory is sub-divided into directories that contain parquet files of only a single SECTOR_ID.

In [2]:
ls -l /kellogg/data/numerator/data/parquet-part/standard_nmr_feed_item_table

total 353
drwxrwx--x+ 2 wkt406 wkt406  4096 Jun 30 12:58 SECTOR_ID=/
drwxrwx--x+ 2 wkt406 wkt406 16384 Jun 30 12:58 SECTOR_ID=isc_apparel/
drwxrwx--x+ 2 wkt406 wkt406 16384 Jun 30 12:58 SECTOR_ID=isc_automotive/
drwxrwx--x+ 2 wkt406 wkt406 16384 Jun 30 12:58 SECTOR_ID=isc_baby/
drwxrwx--x+ 2 wkt406 wkt406 16384 Jun 30 12:58 SECTOR_ID=isc_books/
drwxrwx--x+ 2 wkt406 wkt406 16384 Jun 30 12:58 SECTOR_ID=isc_electronics/
drwxrwx--x+ 2 wkt406 wkt406 16384 Jun 30 12:58 SECTOR_ID=isc_entertainment/
drwxrwx--x+ 2 wkt406 wkt406 16384 Jun 30 12:58 SECTOR_ID=isc_grocery/
drwxrwx--x+ 2 wkt406 wkt406 16384 Jun 30 12:58 SECTOR_ID=isc_gro_household/
drwxrwx--x+ 2 wkt406 wkt406 16384 Jun 30 12:58 SECTOR_ID=isc_gro_tobacco_products_and_accessories/
drwxrwx--x+ 2 wkt406 wkt406 16384 Jun 30 12:58 SECTOR_ID=isc_health_and_beauty/
drwxrwx--x+ 2 wkt406 wkt406 16384 Jun 30 12:58 SECTOR_ID=isc_home_and_garden/
drwxrwx--x+ 2 wkt406 wkt406 16384 Jun 30 12:58 SECTOR_ID=isc_indeterminate_category/
drwxrwx--x+ 2 w

## DuckDB

![DuckDB_features](images/duckdb_features.png)

The main advantage of DuckDB over AWS/Athena is that it is free! You can also use it to build your own databases. It can work on any several different types of files (i.e., CSV, JSON, parquet, compressed/uncompressed, etc.).

Note: all the data used to build Kellogg AWS/Athena databases are available on KLC as parquet files.

### DuckDB/Python with Numerator

Duck DB can be used with Pandas to generate output in dataframes using the .fetchdf() method. You can specify the number of threads used in the query with the SET command.

In [1]:
import duckdb
import pandas as pd
pd.set_option('display.max_columns', None)

In [9]:
query = ''' 
SET threads to 16;
SELECT SECTOR_ID, COUNT(*) AS N
FROM read_parquet('/kellogg/data/numerator/data/parquet-part/standard_nmr_feed_item_table/*/*.parquet',
                    hive_partitioning=true)
GROUP BY SECTOR_ID;
'''
df = duckdb.query(query).fetchdf()

In [10]:
df

,SECTOR_ID,N
0,isc_qsr,3730900
1,isc_toys,2986349
2,isc_indeterminate_category,548714
3,isc_sports,1067313
4,isc_tools_and_home_improvement,2717426
5,na,104810505
6,isc_non_item,3120410
7,isc_office,1559704
8,isc_apparel,10987294
9,isc_automotive,1270808


### Query partitioned database of parquet files

Use the glob string to designate the path to all parquet files in a table. Set the parameter hive_partition = true so that the function recognizes that the file system is set up for partitioning.

The following query uses three tables in the numerator dataset to look up some demographics of consumers who purchased the Lego Horizon Forbidden West Tallneck set.

Let's break down this query:
- The items table (standard_nmr_feed_item_table) has a listing of all products purchased by consumers. It is partitioned by SECTOR_ID, so we make sure to include that field in the filter to only select toys. We also included the DEPT_ID to filter on "building" toys, the brand "Lego", and the keyword "tallneck" in the name of the set included in the description.
- We join with the facts table (standard_nmr_feed_fact_table), which links each purchase of an item (ITEM_ID) with a consumer that purchased it (USER_ID).
- We join a third table people (standard_nmr_feed_people_table) to the facts table on USER_ID to get the demographics of each individual consumer. Note, this people table is also partitioned by census regions; however, even though we aren't interested in the partitions, we still have to indicate that the file system is partitioned.
- We select only the demographic fields we are interested in: gender, age, marital status, and whether or not they have children.
- We can compress the output by asking for counts of each consumer breakout in an aggregate function (COUNT, GROUP BY) rather than listing every person.

In [54]:
query = ''' 
SELECT people.GENDER_APP_USER, people.AGE_BUCKET, people.MARITAL_STATUS, people.HAS_CHILDREN, COUNT(*) AS N
FROM read_parquet('/kellogg/data/numerator/data/parquet-part/standard_nmr_feed_item_table/*/*.parquet',
                hive_partitioning=true) items
JOIN read_parquet('/kellogg/data/numerator/data/parquet-part/standard_nmr_feed_fact_table/*.parquet') facts
ON items.ITEM_ID = facts.ITEM_ID
JOIN read_parquet('/kellogg/data/numerator/data/parquet-part/standard_nmr_feed_people_table/*/*/*.parquet',
                hive_partitioning=true) people
ON people.USER_ID = facts.USER_ID
WHERE items.SECTOR_ID = 'isc_toys' 
AND items.DEPT_ID = 'isc_toy_building'
AND items.BRAND ILIKE '%lego%'
AND items.ITEM_DESCRIPTION ILIKE '%tallneck%'
GROUP BY people.GENDER_APP_USER, people.AGE_BUCKET, people.MARITAL_STATUS, people.HAS_CHILDREN
'''
df = duckdb.sql(query).fetchdf()

In [55]:
df

,GENDER_APP_USER,AGE_BUCKET,MARITAL_STATUS,HAS_CHILDREN,N
0,Male,55-64,Married,No,3
1,Male,35-44,Living with partner,No,5
2,Female,45-54,Divorced,Yes,6
3,Male,35-44,Married,No,6
4,Male,18-20,Never married,Yes,1
...,...,...,...,...,...
62,Female,25-34,Never married,No,8
63,Female,35-44,Living with partner,No,5
64,Female,25-34,Divorced,Yes,1
65,Female,21-24,Married,Yes,1


This query takes about 10 minutes to run on KLC using the maximum number of threads. We can run the same query on Athena and get the same results in about 30 seconds.

![athena](images/athena-duckdb.png)

### Why DuckDB if Athena is faster?

Duck DB is completely free! If you have a very heavy workload, you can utilize DuckDB to run queryies on KLC to and easily integrate the results into your R or Python code. It also serves as a good playground for testing queries before using Athena.

As shown, you can also use DuckDB to query any organized file system that may not be available on Athena (CSV, JSON, etc.), so you could create your own databases for any data specific to your research.